In [1]:
%pip install comtradeapicall

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#%pip install -r requirements.txt

In [3]:
import os
import shutil
import gzip
from pandas import json_normalize
import urllib3
import json
import comtradeapicall

In [4]:
proxies = {
    'http': 'SECRET',
    'https': 'SECRET/'
}
proxy_url = proxies.get('http') 

In [5]:
# API key
subscription_key = 'SECRET'

# Where do you want to save?
directory = '' #'SECRET'

In [6]:
def bulkDownloadFile(subscription_key, directory, tradeDataType, typeCode, freqCode, clCode, period, reporterCode,
                     decompress, publishedDateFrom=None, publishedDateTo=None, proxy_url=proxy_url):

    if tradeDataType == 'TARIFFLINE':
        baseURLDataAvailability = 'https://comtradeapi.un.org/bulk/v1/getTariffline/' + \
            typeCode + '/' + freqCode + '/' + clCode
        prefixFile = 'TARIFFLINE'
    elif tradeDataType == "FINALCLASSIC":
        baseURLDataAvailability = 'https://comtradeapi.un.org/bulk/v1/getClassic/' + \
            typeCode + '/' + freqCode + '/' + clCode
        prefixFile = 'FINALCLASSIC'
    else:
        baseURLDataAvailability = 'https://comtradeapi.un.org/bulk/v1/get/' + \
            typeCode + '/' + freqCode + '/' + clCode
        prefixFile = 'FINAL'

    PARAMS = dict(reportercode=reporterCode, period=period,
                  publishedDateFrom=publishedDateFrom, publishedDateTo=publishedDateTo)
    # add key
    PARAMS["subscription-key"] = subscription_key
    fields = dict(filter(lambda item: item[1] is not None, PARAMS.items()))
    if proxy_url:
        http = urllib3.ProxyManager(proxy_url=proxy_url)
    else:
        http = urllib3.PoolManager()
    try:
        resp = http.request("GET", baseURLDataAvailability,
                            fields=fields, timeout=120)
        if resp.status != 200:
            print(resp.data.decode('utf-8'))
        else:
            jsonResult = json.loads(resp.data)
            if jsonResult['count'] == 0:
                print('No data available based on the selection criteria')
            else:
                # Results contain the required data
                df = json_normalize(jsonResult['data'])
                totalFiles = df[df.columns[0]].count()
                i = 0
                while i < totalFiles:
                    # prepare file name
                    if tradeDataType == 'TARIFFLINE':
                        if (df.timestamp[i] is None):
                            timestamp = '1900-01-01'
                        else:
                            timestamp = df.timestamp[i][:10]
                        fileName = "COMTRADE-" + prefixFile + "-" + df.typeCode[i] + "-" + df.freqCode[i] + "-" + str(df.reporterCode[i]).zfill(
                            3) + "-" + str(df.period[i]) +"-" +  df.classificationCode[i] + ".gz"
                    else:
                        if (df.publicationDate[i] is None):
                            publicationDate = '1900-01-01'
                        else:
                            publicationDate = df.publicationDate[i][:10]
                        fileName = "COMTRADE-" + prefixFile + "-" + df.typeCode[i] +"-" + df.freqCode[i] + "-" + str(
                            df.reporterCode[i]).zfill(
                            3) + "-" + str(df.period[i]) + "-" + df.classificationCode[i] + ".gz"
                    download_path = os.path.join(directory, fileName)
                    # download file
                    file_url = df.fileUrl[i]
                    PARAMS = dict()
                    PARAMS["subscription-key"] = subscription_key
                    fieldsFILE = dict(
                        filter(lambda item: item[1] is not None, PARAMS.items()))
                    if proxy_url:
                        httpFILE = urllib3.ProxyManager(proxy_url=proxy_url)
                    else:
                        httpFILE = urllib3.PoolManager()
                    with open(download_path, 'wb') as out:
                        r = httpFILE.request(
                            'GET', file_url, fields=fieldsFILE, preload_content=False)
                        shutil.copyfileobj(r, out)
                    r.release_conn()
                    print(fileName.replace(".gz", "") + ' downloaded')
                    download_path_gunzip = download_path.replace(".gz", ".txt")
                    if decompress is True:
                        with gzip.open(download_path, "rb") as f_in:
                            with open(download_path_gunzip, 'wb') as f_out:
                                shutil.copyfileobj(f_in, f_out)
                        os.remove(download_path)
                    i = i + 1
                print('Total of ' + str(i) + ' file(s) downloaded')
    except urllib3.exceptions.RequestError as err:
        print(f'Request error: {err}')

In [7]:
def EIB_bulkDownloadFinalFile(subscription_key, directory, typeCode, freqCode, clCode, period=None, reporterCode=None, decompress=False, publishedDateFrom=None, publishedDateTo=None):
    bulkDownloadFile(subscription_key, directory, 'FINAL', typeCode, freqCode, clCode, period,
                     reporterCode, decompress, publishedDateFrom, publishedDateTo)


def EIB_bulkDownloadFinalClassicFile(subscription_key, directory, typeCode, freqCode, clCode, period=None, reporterCode=None, decompress=False, publishedDateFrom=None, publishedDateTo=None):
    bulkDownloadFile(subscription_key, directory, 'FINALCLASSIC', typeCode, freqCode, clCode, period,
                     reporterCode, decompress, publishedDateFrom, publishedDateTo)


def EIB_bulkDownloadTarifflineFile(subscription_key, directory, typeCode, freqCode, clCode, period=None, reporterCode=None, decompress=False, publishedDateFrom=None, publishedDateTo=None):
    bulkDownloadFile(subscription_key, directory, 'TARIFFLINE', typeCode, freqCode, clCode, period,
                     reporterCode, decompress, publishedDateFrom, publishedDateTo)

mydf=EIB_bulkDownloadFinalClassicFile(subscription_key, directory, typeCode='C', freqCode='A', clCode='HS',
                                      period='2020', reporterCode=None, decompress=True)

In [8]:
#mydf = EIB_bulkDownloadFinalClassicFile(subscription_key, directory,typeCode='C', freqCode='A', clCode='HS',
                                  #      period='2023', reporterCode='380', partnerCode = '0', decompress=True)


In [9]:
import pandas as pd

In [10]:
# Define the file path
file_path = 'COMTRADE-FINALCLASSIC-C-A-380-2023-H6.txt'

# Read the text file into a pandas DataFrame
df = pd.read_csv(file_path, delimiter='\t')

: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter='\t')


In [11]:
import pandas
def getPreviewData(subscription_key, tradeDataType, typeCode, freqCode, clCode, period, reporterCode, cmdCode,
                   flowCode,
                   partnerCode,
                   partner2Code, customsCode, motCode, maxRecords, format_output, aggregateBy,
                   breakdownMode,
                   countOnly, includeDesc, proxy_url=proxy_url):
    if subscription_key is not None:
        if tradeDataType == 'TARIFFLINE':
            baseURL = 'https://comtradeapi.un.org/data/v1/getTariffline/' + \
                typeCode + '/' + freqCode + '/' + clCode
        else:
            baseURL = 'https://comtradeapi.un.org/data/v1/get/' + \
                typeCode + '/' + freqCode + '/' + clCode
    else:
        if tradeDataType == 'TARIFFLINE':
            baseURL = 'https://comtradeapi.un.org/public/v1/previewTariffline/' + \
                typeCode + '/' + freqCode + '/' + clCode
        else:
            baseURL = 'https://comtradeapi.un.org/public/v1/preview/' + \
                typeCode + '/' + freqCode + '/' + clCode

    PARAMS = dict(reportercode=reporterCode, flowCode=flowCode,
                  period=period, cmdCode=cmdCode, partnerCode=partnerCode, partner2Code=partner2Code,
                  motCode=motCode, customsCode=customsCode,
                  maxRecords=maxRecords, format=format_output, aggregateBy=aggregateBy, breakdownMode=breakdownMode,
                  countOnly=countOnly, includeDesc=includeDesc)
    PARAMS["subscription-key"] = subscription_key
    fields = dict(filter(lambda item: item[1] is not None, PARAMS.items()))
    if proxy_url:
        http = urllib3.ProxyManager(proxy_url=proxy_url)
    else:
        http = urllib3.PoolManager()
    if format_output is None:
        format_output = 'JSON'
    if format_output != 'JSON':
        print("Only JSON output is supported with this function")
    else:
        try:
            resp = http.request("GET", baseURL, fields=fields, timeout=120)
            if resp.status != 200:
                print(resp.data.decode('utf-8'))
            else:
                jsonResult = json.loads(resp.data)
                if countOnly:
                    dictCount = dict(count=jsonResult['count'])
                    df = pandas.DataFrame([dictCount])
                else:
                    # Results contain the required data
                    df = json_normalize(jsonResult['data'])
                return df
        except urllib3.exceptions.RequestError as err:
            print(f'Request error: {err}')


def previewFinalData(typeCode, freqCode, clCode, period, reporterCode, cmdCode, flowCode,
                     partnerCode,
                     partner2Code, customsCode, motCode, maxRecords=None, format_output=None,
                     aggregateBy=None, breakdownMode=None, countOnly=None, includeDesc=None, proxy_url=proxy_url):
    return getPreviewData(None, 'FINAL', typeCode, freqCode, clCode, period, reporterCode,
                          cmdCode, flowCode,
                          partnerCode,
                          partner2Code, customsCode, motCode, maxRecords, format_output, aggregateBy,
                          breakdownMode,
                          countOnly, includeDesc, proxy_url)


def _previewFinalData(typeCode, freqCode, clCode, period, reporterCode, cmdCode, flowCode,
                      partnerCode,
                      partner2Code, customsCode, motCode, maxRecords=None, format_output=None,
                      aggregateBy=None, breakdownMode=None, countOnly=None, includeDesc=None, proxy_url=proxy_url):
    main_df = pandas.DataFrame()
    for single_period in list(period.split(",")):
        try:
            staging_df = previewFinalData(typeCode, freqCode, clCode, single_period, reporterCode, cmdCode, flowCode,
                                          partnerCode,
                                          partner2Code, customsCode, motCode, maxRecords, format_output, aggregateBy,
                                          breakdownMode,
                                          countOnly, includeDesc, proxy_url)
        except:  # retry once more after 10 secs  # noqa: E722
            print('Repeating API call for period: ' + single_period)
            t.sleep(10)
            staging_df = previewFinalData(typeCode, freqCode, clCode, single_period, reporterCode, cmdCode, flowCode,
                                          partnerCode,
                                          partner2Code, customsCode, motCode, maxRecords, format_output, aggregateBy,
                                          breakdownMode,
                                          countOnly, includeDesc)
        main_df = pandas.concat([main_df, staging_df])
    return main_df


def previewTarifflineData(typeCode, freqCode, clCode, period, reporterCode, cmdCode, flowCode,
                          partnerCode,
                          partner2Code, customsCode, motCode, maxRecords=None, format_output=None,
                          countOnly=None, includeDesc=None, proxy_url=proxy_url):
    return getPreviewData(None, 'TARIFFLINE', typeCode, freqCode, clCode, period, reporterCode,
                          cmdCode, flowCode,
                          partnerCode,
                          partner2Code, customsCode, motCode, maxRecords, format_output, None,
                          None,
                          countOnly, includeDesc, proxy_url)


def _previewTarifflineData(typeCode, freqCode, clCode, period, reporterCode, cmdCode, flowCode,
                           partnerCode,
                           partner2Code, customsCode, motCode, maxRecords=None, format_output=None,
                           countOnly=None, includeDesc=None, proxy_url=proxy_url):
    main_df = pandas.DataFrame()
    for single_period in list(period.split(",")):
        try:
            staging_df = previewTarifflineData(typeCode, freqCode, clCode, single_period, reporterCode, cmdCode,
                                               flowCode,
                                               partnerCode,
                                               partner2Code, customsCode, motCode, maxRecords, format_output,
                                               countOnly, includeDesc, proxy_url)
        except:  # retry once more after 10 secs  # noqa: E722
            print('Repeating API call for period: ' + single_period)
            t.sleep(10)
            staging_df = previewTarifflineData(typeCode, freqCode, clCode, single_period, reporterCode, cmdCode, flowCode,
                                               partnerCode,
                                               partner2Code, customsCode, motCode, maxRecords, format_output,
                                               countOnly, includeDesc)
        main_df = pandas.concat([main_df, staging_df])
    return main_df


def getFinalData(subscription_key, typeCode, freqCode, clCode, period, reporterCode, cmdCode, flowCode,
                 partnerCode,
                 partner2Code, customsCode, motCode, maxRecords=None, format_output=None,
                 aggregateBy=None, breakdownMode=None, countOnly=None, includeDesc=None, proxy_url=proxy_url):
    return getPreviewData(subscription_key, 'FINAL', typeCode, freqCode, clCode, period, reporterCode,
                          cmdCode, flowCode,
                          partnerCode,
                          partner2Code, customsCode, motCode, maxRecords, format_output, aggregateBy,
                          breakdownMode,
                          countOnly, includeDesc, proxy_url)


def _getFinalData(subscription_key, typeCode, freqCode, clCode, period, reporterCode, cmdCode, flowCode,
                  partnerCode,
                  partner2Code, customsCode, motCode, maxRecords=None, format_output=None,
                  aggregateBy=None, breakdownMode=None, countOnly=None, includeDesc=None, proxy_url=proxy_url):
    main_df = pandas.DataFrame()
    for single_period in list(period.split(",")):
        try:
            staging_df = getFinalData(subscription_key, typeCode, freqCode, clCode, single_period, reporterCode,
                                      cmdCode,
                                      flowCode, partnerCode,
                                      partner2Code, customsCode, motCode, maxRecords, format_output, aggregateBy,
                                      breakdownMode,
                                      countOnly, includeDesc, proxy_url)
        except:  # retry once more after 10 secs  # noqa: E722
            print('Repeating API call for period: ' + single_period)
            t.sleep(10)
            staging_df = getFinalData(subscription_key, typeCode, freqCode, clCode, single_period, reporterCode, cmdCode,
                                      flowCode, partnerCode,
                                      partner2Code, customsCode, motCode, maxRecords, format_output, aggregateBy,
                                      breakdownMode,
                                      countOnly, includeDesc, proxy_url)
        main_df = pandas.concat([main_df, staging_df])
    return main_df


def getTarifflineData(subscription_key, typeCode, freqCode, clCode, period, reporterCode, cmdCode, flowCode,
                      partnerCode,
                      partner2Code, customsCode, motCode, maxRecords=None, format_output=None,
                      countOnly=None, includeDesc=None, proxy_url=proxy_url):
    return getPreviewData(subscription_key, 'TARIFFLINE', typeCode, freqCode, clCode, period, reporterCode,
                          cmdCode, flowCode,
                          partnerCode,
                          partner2Code, customsCode, motCode, maxRecords, format_output, None,
                          None,
                          countOnly, includeDesc, proxy_url)


def _getTarifflineData(subscription_key, typeCode, freqCode, clCode, period, reporterCode, cmdCode, flowCode,
                       partnerCode,
                       partner2Code, customsCode, motCode, maxRecords=None, format_output=None,
                       countOnly=None, includeDesc=None, proxy_url=proxy_url):
    main_df = pandas.DataFrame()
    for single_period in list(period.split(",")):
        try:
            staging_df = getTarifflineData(subscription_key, typeCode, freqCode, clCode, single_period, reporterCode,
                                           cmdCode, flowCode, partnerCode,
                                           partner2Code, customsCode, motCode, maxRecords, format_output,
                                           countOnly, includeDesc, proxy_url)
        except:  # retry once more after 10 secs  # noqa: E722
            print('Repeating API call for period: ' + single_period)
            t.sleep(10)
            staging_df = getTarifflineData(subscription_key, typeCode, freqCode, clCode, single_period, reporterCode,
                                           cmdCode, flowCode, partnerCode,
                                           partner2Code, customsCode, motCode, maxRecords, format_output,
                                           countOnly, includeDesc, proxy_url)
        main_df = pandas.concat([main_df, staging_df])
    return main_df

In [12]:
#get the top 4 digits exported commodities
#classic breakdown mode will set the partner2Code to World, customsCode to Total, and motCode to Total.
df = _getFinalData(subscription_key, typeCode='C', freqCode='A', clCode='HS',period="2023",reporterCode='380',cmdCode='AG2', flowCode="X", partnerCode=0, partner2Code=None, customsCode=None,motCode=None,breakdownMode='classic', includeDesc=True)

In [13]:
A = df["primaryValue"].sum()
A

677095229000.93

In [15]:
dftot = _getFinalData(subscription_key, typeCode='C', freqCode='A', clCode='HS',period="2022",reporterCode=None,cmdCode='AG2', flowCode="X", partnerCode=0, partner2Code=None, customsCode=None,motCode=None,breakdownMode='classic', includeDesc=True)

In [19]:
reporter_codes = dftot["reporterCode"].unique()
years = range(2005, 2024)  # From 2017 to 2023
# List to hold all DataFrames

for year in years:
    data_frames = []

    # Loop through reporter codes and download the data
    for code in reporter_codes:
        df = _getFinalData(subscription_key, typeCode='C', freqCode='A', clCode='HS',period=str(year),reporterCode= code,cmdCode='AG2', flowCode="X", partnerCode=0, partner2Code=None, customsCode=None,motCode=None,breakdownMode='classic', includeDesc=True)
        data_frames.append(df)

    # Concatenate all DataFrames
    final_df = pd.concat(data_frames, ignore_index=True)

    final_df.to_excel(f'AG2_comtrade_data_{year}.xlsx', index=False)
    
    # Optional: Print a confirmation message
    print(f'Data for {year} has been saved to comtrade_data_{year}.xlsx')

14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(data_frames, ignore_index=True)


Data for 2005 has been saved to comtrade_data_2005.xlsx


:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(data_frames, ignore_index=True)


Data for 2006 has been saved to comtrade_data_2006.xlsx


:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(data_frames, ignore_index=True)


...

Data for 2020 has been saved to comtrade_data_2020.xlsx


:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(data_frames, ignore_index=True)


Data for 2021 has been saved to comtrade_data_2021.xlsx


:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(data_frames, ignore_index=True)


Data for 2022 has been saved to comtrade_data_2022.xlsx
Data for 2023 has been saved to comtrade_data_2023.xlsx
